In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [2]:
device = torch.device("cuda" if torch.cuda.is_available else "cpu")
device

device(type='cuda')

In [3]:
def odd_even(x):
    return (x, 1.0) if x % 2 == 0 else (x, 0.0)

In [4]:
data = np.array(list(map(odd_even, range(0, 10000))))

In [5]:
training_data, testing_data = train_test_split(data, test_size=0.2)

In [6]:
train_loader = DataLoader(training_data, batch_size=64, shuffle=True)
test_loader = DataLoader(testing_data, batch_size=64, shuffle=True)

In [7]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = self.get_fc_layers()
    
    def get_fc_layers(self):
        layers = nn.Sequential(
            nn.Linear(1, 8), 
            nn.Dropout(0.1), 
            nn.Linear(8, 64),
            nn.ReLU(),
            nn.Linear(64, 512), 
            nn.Linear(512, 128), 
            nn.Linear(128, 16), 
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid())
        return layers
    
    def forward(self, x):
        x = self.layers(x)
        return x

In [8]:
net = Net()
net.to(device)

Net(
  (layers): Sequential(
    (0): Linear(in_features=1, out_features=8, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): Linear(in_features=8, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=512, bias=True)
    (5): Linear(in_features=512, out_features=128, bias=True)
    (6): Linear(in_features=128, out_features=16, bias=True)
    (7): ReLU()
    (8): Linear(in_features=16, out_features=1, bias=True)
    (9): Sigmoid()
  )
)

In [13]:
# Defining our Loss Function
criterion = nn.MSELoss()

# Defining optimizer
optimizer = optim.SGD(net.parameters(), lr=0.003, momentum=0.9)

# Number of epochs
epochs = 15
for epoch in range(epochs):

    # Initialising statistics that we will be tracking across epochs
    total_correct = 0
    total = 0
    total_loss = 0

    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        feature, label = data[:, 0], data[:, 1]
        # loading onto cuda if available*
        if torch.cuda.is_available():
            feature, label = feature.to(torch.float), label.to(torch.float)
            feature = feature.to(device)
            label = label.to(device)

        
        # zero the parameter gradients: Clean the gradient caclulated in the previous iteration
        optimizer.zero_grad() # Set all graidents to zero for each step as they accumulate over backprop

        # forward + backward + optimize
        feature = feature.view(feature.shape[0], -1)
        output = net.forward(feature)
        loss = criterion(output.reshape(-1), label)

        # Calculate gradient of matrix with requires_grad = True
        loss.backward() #computes dloss/dx for every parameter x which has requires_grad=True

        # Apply the gradient calculate from last step to the matrix
        optimizer.step() # x += -lr * x.grad ie updates the weights of the parameters

        # Adding loss to total loss
        total_loss += loss.item()

        # Checking which output label has max probability
        _, predicted = torch.max(output, 1)
        total += label.shape[0]

        # Tracking number of correct predictions
        total_correct += (predicted == label).sum().item()

    # Calculating accuracy, epoch-time+-
    accuracy = 100* total_correct/total

    # Printing out statistics
    print("Epoch no.",epoch+1 ,"|accuracy: ", round(accuracy, 3),"%", "|total_loss: ", total_loss)

Epoch no. 1 |accuracy:  49.9 % |total_loss:  62.61319017410278
Epoch no. 2 |accuracy:  49.9 % |total_loss:  62.61124449968338
Epoch no. 3 |accuracy:  49.9 % |total_loss:  62.612939804792404
Epoch no. 4 |accuracy:  49.9 % |total_loss:  62.61216792464256
Epoch no. 5 |accuracy:  49.9 % |total_loss:  62.61262181401253
Epoch no. 6 |accuracy:  49.9 % |total_loss:  62.61132237315178
Epoch no. 7 |accuracy:  49.9 % |total_loss:  62.612674593925476
Epoch no. 8 |accuracy:  49.9 % |total_loss:  62.6118146777153
Epoch no. 9 |accuracy:  49.9 % |total_loss:  62.61149114370346
Epoch no. 10 |accuracy:  49.9 % |total_loss:  62.6122100353241
Epoch no. 11 |accuracy:  49.9 % |total_loss:  62.61246970295906
Epoch no. 12 |accuracy:  49.9 % |total_loss:  62.61217746138573
Epoch no. 13 |accuracy:  49.9 % |total_loss:  62.60938674211502
Epoch no. 14 |accuracy:  49.9 % |total_loss:  62.611732602119446
Epoch no. 15 |accuracy:  49.9 % |total_loss:  62.611789256334305
